<h1>Overfitting and Generalization Bounds</h1>

Suppose you want to write a program that predicts how much rain there will be tomorrow from 1000 different types of weather data recorded today (average wind-speed, barometric pressure, percent cloud cover, temperature, etc
). You aren't a meteorologist though, so you don't know anything about how the rain should actually be influenced by these variables. Instead, you're going to apply this new-fangled machine learning stuff that you've heard so much about. You remember from physics class that all objects are point masses and all functions are linear, so you decide that the amount of precipitation probably can be written as
$$
r = a_1x_1+a_2x_2+\cdots+a_{1000} x_{1000}
$$
where here $r$ is the amount of precipitation (say in inches), $x_1,\dots,x_{1000}$ are your weather measurements, and $a_1,\dots,a_{1000}$ are some constants that you don't know. All you have to do is find the values of $a_i$, and then you're done!

In order to do this, you record weather data for 1000 days (maybe you did this in 50 places for 20 days each, or maybe you spent 3 years, whatever floats your boat), and use a nice easy optimization package to find values for $a_i$ that predict your recorded data well. You find that you can predict your collected data perfectly! This is extremely exciting, but before rushing to start a weather-prediction company, you decide to check if your predictions hold up. So you collect data for 2000 more days, and check how well your predictions match up with reality. This time, you discover to your chagrin that your predictions are horrible! 

So what happened? The problem, of course, is <i>overfitting</i>. By minimizing your error on your training set (the 1000 days of training data), you adapted too strongly to the peculiarities of that dataset. Intuitively, your linear function is also predicting the noise that's present in your 1000 observations. It continued to try to add this noise in when making predictions on the 2000 new observations, and so ended up with very poor predictions because the noise is actually irrelevant. This intuition may also give you some idea about how to deal with this. One way is to simply collect more data to start with, so that it's easier for your linear function to identify the signal in the noise. But what if collecting data is expensive and you only have money for 1000 training measurements? Then another approach would be to make the linear model <i>less powerful</i>. The problem here was that you were able to perfectly model the noise in your data. If you instead restricted yourself to only considering the first 100 measurements (for example), perhaps it would be impossible to model the noise properly and so you wouldn't overfit. On the other hand, it might be that by ignoring the last 900 observations you make it impossible to find a good predictor for the rain. This leads to a natural trade-off between the <i>capacity</i> of your model (the degree to which it could predict the rain if trained correctly), and the <i>generalization error</i> (the degree to which your model overfits to the training set).


<h2>Some Mathematical Notation</h2>
$\newcommand{\E}{\mathop{\mathbb{E}}}\newcommand{\L}{\mathcal{L}}\newcommand{\H}{\mathcal{H}}\newcommand{\R}{\mathbb{R}}$
Let's make our setting precise so that we can actually prove some things. In general, we are trying to find some function $h$ that "performs well" on an observed datapoint $z$. We'll measure the degree to which $h$ performs well with a loss function $\L(h,z)$. More formally, let $Z$ be a space of possible observations (e.g. weather data and rain measurements), $\H$ be a space of possible functions (e.g. our linear functions), and $\L$ a function from $\H\times Z\to \R$. Let $D$ be some unknown distribution over $Z$. Then our objective is to find a function $h^\star\in \H$ that minimizes $\L(h)=\E_{z\sim D}[L(h,z)]$. The function $\L$ is called the <i>risk</i>.

In our weather example, $Z=X\times \R$ where $X$ is the space of tuples of measured weather data $(x_1,\dots,x_{1000})$ and a data point $(x,r)$ indicates that $r$ inches of rain were observed the day after recording weather data $x$. $\H$ is the set of linear functions $X\to \R$, and we might use the loss function $\L(h,(x,r)) = \frac{1}{2}(h(x)-r)^2$.

This problem is made much more difficult by the fact that we don't know the distribution $D$. Instead, we have a training set, which is a sample of $N$ independent points $z_1,\dots,z_N$ distributed according to $D$. How can we use this training set to find an $h$ with small $\L(h)$? One natural approach is <i>empirical risk minimization</i>, which is just a fancy way of saying "minimize your training error". Formally, we define the <i>empirical risk</i> (a.k.a. training error) by $\hat \L(h)=\frac{1}{N}\sum_{i=1}^N \L(h,z_i)$. Then we can hope that $\hat \L(h)$ is a good approximation for $\L(h)$, and instead find $\hat h$ that minimizes $\hat \L$. This is a pretty reasonable-sounding thing to do, since in general averages should converge to their expectated values.

<h2>Hoeffding's Bound</h2>
In fact, so long as $\L(h,z_i)$ isn't allowed to be too big (e.g. if $\L(h,z_i)\in[0,B]$ with probability 1 for some constant $B$), then <a href=https://en.wikipedia.org/wiki/Hoeffding_inequality>Hoeffding's inequality</a> tells us that for any $h\in \H$,
$$
\Pr\left[\left| \L(h)-\hat \L(h)\right|>\epsilon\right]=\Pr\left[\left| \L(h)-\frac{1}{N}\sum_{i=1}^N \L(h,z_i)\right|>\epsilon\right] \le 2\exp\left(\frac{-2N\epsilon^2}{B^2}\right)
$$
This says that the average value, $\frac{1}{N}\sum_{i=1}^N \L(h,z_i)=\hat \L(h)$ is almost certainly very very close to the expected value $\L(h)$. In particular, the probability that the difference between the average and the expectation is greater than $\frac{2B}{\sqrt{2N}}$ is less than $0.04$. In general, with probability at least $1-\delta$, we have
$$
\L(\hat h) \le \hat \L(h) + \frac{B\sqrt{\log(2/\delta)}}{\sqrt{2N}}
$$

This calculation makes it seem like $\hat \L$ is a great approximation for $\L$, and so $\hat h$ must be a really great approximation for the best function $h^\star$. If we have $B=1$ (which in our setting corresponds to a loss function that is bounded between 0 and 1, like the 0-1 classification loss), then for $N=1000$ we should expect an error of less than $0.05$!

This seems very promising, but unfortunately life is not so easy. If you're not already suspicious, here's an example illustrating that something is wrong:

<div style="margin-left: 3em; margin-right: 6em;" class="aside">
<p>
Suppose you are fitting the weather-prediction problem again, but now you only care to predict whether or not there will be rain. You're going to again use a linear model $r = a_1x_1+a_2x_2+\cdots+a_{1000} x_{1000}$ and output "rain" if $r>0$ and "no rain" if $r\le 0$. The loss will be just be 1 if you predict correctly and 0 otherwise. This loss is bounded between 0 and 1, so $B=1$ and by the above argument you'd expect the empirical risk $\hat L$ to be only 0.05 away from the true risk $\L$. However, suppose that your 1000 weather measurements are really just noise - someone stole all your recording equipment and replaced them with random number generators. Then you should still expect to find an $\hat h$ with $\hat \L(\hat h)=0$ because you have a linear system with 1000 variables (the $a_i$'s) and 1000 equations (the 1000 training observations), so you can always find some $a_i$'s that will match the training points perfectly. However, since your actual measurements $x_i$ are garbage, it's silly to expect that the true best predictor would have expected loss of only 0.05.
</p>
</div>

So what went wrong? Essentially, we've run afoul of a multiple-hypothesis testing issue. Our Hoeffding bound equation holds individually for each $h\in \H$, but it does not hold for all values of $h$ <i>at the same time</i>. This can be kind of a tricky concept, so here's another example of what's happening.:

<div style="margin-left: 3em; margin-right: 6em;" class="aside">
<p>
Suppose you and $1000$ friends each toss a biased coin that lands heads with probability $0.999$. Then you should be almost positive that your coin will land heads. However, it's also pretty likely that at least one of your friends' coins landed tails. Mathematically, the probability of none of your friends getting a toss of tails is $(0.999)^{1000}\le 0.37$. Exactly the same thing is happening in our empirical risk minimization problem. The "friends" are the functions $h\in \H$. Each individual function $h$ has training error that's probably quite close to the true error. However, the probability that there is <i>some</i> $h\in \H$ whose training error is very far from the true error is also very high. Since you are selecting the $h$ with the smallest training error, this will effectively bias you towards those $h$'s whose training errors have a big downward deviation from the true error, resulting in overfitting.
</p>
</div>

<h2>The Union Bound</h2>
How can we analyze this problem correctly? Well, the issue with the Hoeffding's inequality argument was that the probability bound holds only for each $h\in H$ individually. Maybe we should instead look for a statement that holds for ALL $h\in H$ simultaneously. This is called the "uniform convergence" approach, and is the basis for most generalization bounds. As a gentle foray into this area, let's assume that $\H$ contains only 10 functions $h_1,\dots,h_{10}$. Then for each function individually, we have
$$
\Pr\left[\left| \L(h_j)-\frac{1}{N}\sum_{i=1}^N \L(h_j,z_i)\right|>\epsilon\right] \le 2\exp\left(\frac{-2N\epsilon^2}{B^2}\right)
$$
Now we use the extremely useful <a href=https://en.wikipedia.org/wiki/union_bound>union bound</a>, which simply says that if there are $M$ "bad" events that occur with probabilities at most $p_1,\dots,p_M$, then the probability that none of the bad events occur is at most $p_1+\cdots+p_M$. Therefore the probability that none of the 10 functions in $\H$ has an average loss that is far from the true loss is at most
$$
\Pr\left[\max_{j\in \{1,\dots,10\}}\left| \L(h_j)-\frac{1}{N}\sum_{i=1}^N \L(h_j,z_i)\right|>\epsilon\right] \le 10\times2\exp\left(\frac{-2N\epsilon^2}{B^2}\right)
$$

More generally, if $\H$ contains $M$ functions, then
$$
\Pr\left[\max_{h\in \H}\left| \L(h)-\frac{1}{N}\sum_{i=1}^N \L(h,z_i)\right|>\epsilon\right] \le 2M\exp\left(\frac{-2N\epsilon^2}{B^2}\right)
$$

If you set the right-hand-side of the above equation to $\delta$ and solve for $\epsilon$, then this tells us that with probability at least $1-\delta$, we have
$$
\max_{h\in \H} \left| \L(h)-\hat \L(h)\right| < \frac{B\sqrt{\log(2M/\delta)}}{\sqrt{2N}}
$$

Since this statement holds for all $h\in \H$ (because of the $\max$), then it also holds for our selected $\hat h$, and so with probability at least $1-\delta$, we have both
$$
\L(\hat h) \le \hat \L(\hat h) + \frac{B\sqrt{\log(2M/\delta)}}{\sqrt{2N}}\\
\hat \L(h^\star) \le \L(h^\star) + \frac{B\sqrt{\log(2M/\delta)}}{\sqrt{2N}}
$$
Now since $ \L(\hat h)\le \hat \L(h^\star)$ because $\hat h$ minimizes $\hat \L$, we have 
$$
\begin{align*}
\hat \L(\hat h) &= \L(h^\star)+ \hat \L(h^\star)- \L(h^\star)+\hat L(\hat h)-\hat \L(h^\star)\\
 &\le \L(h^\star)+\hat \L(h^\star)-\L(h^\star)\\
&\le \L(h^\star) +  \frac{B\sqrt{\log(2M/\delta)}}{\sqrt{2N}}
\end{align*}
$$
so that we can conclude
$$
\begin{align*}
\L(\hat h) &\le \L(h^\star) + 2\frac{B\sqrt{\log(2M/\delta)}}{\sqrt{2N}}\\
&=\L(h^\star) + \frac{B\sqrt{2\log(2M/\delta)}}{\sqrt{N}}
\end{align*}
$$
If you go back and compare to the statement we got with just Hoeffding's inequality without the union bound, the only difference is the additional $M$ inside the logarithm. This seems pretty nice - we're only paying a logarithmic cost to pick a model out of $M$ possibilities. This sort of observation is at the root of multiple hypothesis testing. Each function $h\in \H$ corresponds to a different hypothesis, and the scientist is picking the hypothesis that best explains the data, which corresponds to having a low $\hat L(h)$. Using this union bound identity allows the scientist to properly compute those $p$-values they are all so fond of.

Unfortunately, we still can't analyze our weather-prediction problem, because the space of linear functions is infinite! Even if the amount of overfitting only depends logarithmically on $M$, $\log(\infty)$ is still $\infty$. So far we've use Hoeffding's bound and the union bound. We need a third ingredient to deal with infinitely large spaces $\H$.

<h2>Covering Numbers</h2>
One way to avoid the problem of not being able to deal with infinite $\H$ is to simply never use an infinite $\H$. For example, in our weather prediction problem we could pick some finite set of linear functions $h_1,\dots,h_M$ before seeing any training data and just find the $h_i$ that has the smallest error on our training set. This would allow us to apply our Hoeffding+Union bound result to bound how much we overfit. Obviously we're losing something by restricting ourselves to only $M$ linear functions, but maybe if we picked a very large $M$, then this would be ok.

This strategy isn't all that unreasonable because if you make a small change to the coefficients $a_i$ of our linear function $a_1x_1+\cdots + a_{1000}x_{1000}$, then the output shouldn't change that much. Specifically, if our weather data satisfies $\sum_{i=1}^{1000} x_i^2 \le L^2$ for some constant $L$, then if we change each $a_i$ by some small amount $\Delta_i$, then by Cauchy-Schwarz inequality, total value of $a_1x_1+\cdots+a_{1000}x_{1000}$ will change by at most $L\sqrt{\sum_{i=1}^{1000} \Delta_i^2}$. This means that if one of our finitely-many choices $h_i$ is "close" to the best choice $h^\star$, then we'll have $\L(h_i)-\L(h^\star)$ is very small, which means that we didn't lose much by selecting only a finite subset of $\H$.

How can we guarantee that $h^\star$ will be close to one of our $h_i$? Maybe we can be super-aggressive and choose enough $h_i$ so that EVERY $h\in \H$ is close to some $h_i$. Unfortunately, this is actually impossible to do with finitely many $h_i$. However, if we restrict $\H$ to be only contain bounded linear functions, then we're back in business. Specifically, let's say that $\H_D$ is the set of linear functions given by tuples $(a_1,\dots,a_{1000})$ such that the norm of this tuple is bounded by $D$. That is, $\sum_{i=1}^{1000} a_i^2\le D^2$. Notice that $\H_D$ is still infinite so this restriction doesn't immediately give us a generalization bound.

We can identify $\H_D$ with vectors of norm at most $D$ in $\R^{1000}$ using the tuples of coefficients $(a_1,\dots,a_{1000})$. Therefore our problem of finding $M$ functions in $\H_D$ such that every point in $\H_D$ is close to one of our $M$ functions is the same as finding $M$ points in $\R^{1000}$ such that every point in the ball of radius $D$ about the origin in $\R^{1000}$ is close to one of our $M$ points. By our Cauchy-Shwarz argument, if $\|h_i-h^\star\|\le \delta$ for some $h_i\in \H_D$, then $\L(h_i)\le \L(h^\star)+\delta L$.